In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
# Import Warnings 
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split
# Import tensorflow as the backend for Keras
from keras import backend as K
K.set_image_dim_ordering('tf')
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD,RMSprop,adam
from keras.callbacks import TensorBoard
# Import required libraries for cnfusion matrix
from sklearn.metrics import classification_report,confusion_matrix
import itertools
####################################################################################################
import keras
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [2]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

#!ls "/content/drive/My Drive/defi_bigdata/output"

#uploaded = files.upload()
y = np.load("/content/drive/My Drive/defi_bigdata/y_64.npy", 'r')
X = np.load("/content/drive/My Drive/defi_bigdata/X_64.npy", 'r')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=123456)
X_valid, X_test, y_valid, y_test = train_test_split(X_valid, y_valid, test_size=0.5, random_state=2)

print("X_train shape = {}".format(X_train.shape))
print("X_valid shape = {}".format(X_valid.shape))

X_train shape = (51163, 64, 64, 3)
X_valid shape = (8527, 64, 64, 3)


In [4]:
batch_size = 256
num_classes = 2

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',padding='same',input_shape=(64,64,3)))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D((2, 2),padding='same'))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), activation='relu',padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), activation='relu',padding='same'))
model.add(LeakyReLU(alpha=0.1))                  
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(LeakyReLU(alpha=0.1))           
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(),metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 64, 64, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 64)        0         
__________

In [11]:
history = model.fit(X_train, y_train, batch_size=batch_size,epochs=10,verbose=1,validation_data=(X_valid, y_valid))

Train on 51163 samples, validate on 8527 samples
Epoch 1/10
51163/51163 [==============================] - 29s 560us/step - loss: 1.1608 - acc: 0.8641 - val_loss: 0.1956 - val_acc: 0.9378
Epoch 2/10
51163/51163 [==============================] - 27s 537us/step - loss: 0.1791 - acc: 0.9373 - val_loss: 0.1391 - val_acc: 0.9549
Epoch 3/10
51163/51163 [==============================] - 27s 536us/step - loss: 0.1533 - acc: 0.9460 - val_loss: 0.1330 - val_acc: 0.9548
Epoch 4/10
51163/51163 [==============================] - 27s 534us/step - loss: 0.1389 - acc: 0.9507 - val_loss: 0.1113 - val_acc: 0.9590
Epoch 5/10
51163/51163 [==============================] - 27s 536us/step - loss: 0.1273 - acc: 0.9549 - val_loss: 0.1125 - val_acc: 0.9601
Epoch 6/10
51163/51163 [==============================] - 27s 535us/step - loss: 0.1233 - acc: 0.9557 - val_loss: 0.1242 - val_acc: 0.9572
Epoch 7/10
51163/51163 [==============================] - 27s 533us/step - loss: 0.1165 - acc: 0.9583 - val_loss: 0.1

In [12]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

acc: 96.10%


In [17]:
from keras.preprocessing.image import ImageDataGenerator
image_gen = ImageDataGenerator(
    #featurewise_center=True,
    #featurewise_std_normalization=True,
    #rescale=1./255,
    rotation_range=90,
    width_shift_range=.2,
    height_shift_range=.2,
    horizontal_flip=True,
    vertical_flip=True)

ind = np.random.choice(X_train.shape[0], len(X_train), replace = False)
X_train2 = X_train[ind]
y_train2 = y_train[ind]

X_train2.shape
X_train.shape

#training the image preprocessing
image_gen.fit(X_train2, augment=True)

result_x  = np.concatenate((X_train, X_train2), axis=0)
result_y  = np.concatenate((y_train, y_train2), axis=0)

result_x.shape
result_y.shape

(102326, 64, 64, 3)

In [0]:
history = model.fit(result_x, result_y, batch_size=batch_size,epochs=10,verbose=1,validation_data=(X_valid, y_valid))

In [19]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

acc: 96.44%


In [0]:
from keras.preprocessing.image import ImageDataGenerator
image_gen = ImageDataGenerator(
    #featurewise_center=True,
    #featurewise_std_normalization=True,
    #rescale=1./255,
    rotation_range=90,
    width_shift_range=.2,
    height_shift_range=.2,
    horizontal_flip=True,
    vertical_flip=True)

image_gen.fit(X_train)
history = model.fit_generator(image_gen.flow(X_train, y_train, batch_size=batch_size), verbose=1,
                        steps_per_epoch=X_train.shape[0] // batch_size,
                        epochs=100,
                        validation_data=(X_valid, y_valid))

In [6]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

acc: 49.03%


### **VGG**

In [0]:
from keras.applications import VGG16
#Load the VGG model
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

# Freeze the layers except the last 4 layers
for layer in vgg_conv.layers[:-4]:
    layer.trainable = False
 
# Check the trainable status of the individual layers
for layer in vgg_conv.layers:
    print(layer, layer.trainable)

from keras import models
from keras import layers
from keras import optimizers
 
# Create the model
model = models.Sequential()
 
# Add the vgg convolutional base model
model.add(vgg_conv)
 
# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='sigmoid'))
 
# Show a summary of the model. Check the number of trainable parameters
model.summary()

# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=1e-4),
              metrics=['acc'])

In [0]:
history = model.fit(X_train, y_train, batch_size=256,epochs=3,verbose=1,validation_data=(X_valid, y_valid))

In [15]:
image_gen.fit(X_train)
history = model.fit_generator(image_gen.flow(X_train, y_train, batch_size=128), verbose=1,
                        steps_per_epoch=X_train.shape[0] // 128,
                        epochs=4,
                        validation_data=(X_valid, y_valid))

Epoch 1/4
399/399 [==============================] - 90s 227ms/step - loss: 0.2643 - acc: 0.9088 - val_loss: 0.1570 - val_acc: 0.9410
Epoch 2/4
399/399 [==============================] - 88s 220ms/step - loss: 0.1565 - acc: 0.9413 - val_loss: 0.1178 - val_acc: 0.9581
Epoch 3/4
399/399 [==============================] - 87s 218ms/step - loss: 0.1432 - acc: 0.9467 - val_loss: 0.1236 - val_acc: 0.9543
Epoch 4/4
399/399 [==============================] - 87s 218ms/step - loss: 0.1293 - acc: 0.9514 - val_loss: 0.1141 - val_acc: 0.9568


In [16]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

acc: 95.22%


### ResNet50

In [18]:
from keras.applications import ResNet50
from keras import models
from keras import layers
from keras import optimizers

model = ResNet50(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
y = model.output
y = Flatten()(y)
y = Dropout(0.5)(y)
y = Dense(2, activation='softmax', name='class_id')(y)
new_model = Model(inputs=model.input, outputs=y)
 
# Show a summary of the model. Check the number of trainable parameters
new_model.summary()

# Compile the model
new_model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=1e-4),
              metrics=['acc'])

#new_model.fit(result_x, result_y, epochs=1, batch_size=128,validation_data=(X_valid,y_valid))

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 70, 70, 3)    0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 32, 32, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 32, 32, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [0]:
image_gen.fit(X_train)
history = new_model.fit_generator(image_gen.flow(X_train, y_train, batch_size=128), verbose=1,
                        steps_per_epoch=X_train.shape[0] // 128,
                        epochs=4,
                        validation_data=(X_valid, y_valid))

In [0]:
scores = new_model.evaluate(X_test, y_test, verbose=0)#97.40% epochs 18
print("%s: %.2f%%" % (new_model.metrics_names[1], scores[1]*100))

In [0]:
from keras.models import model_from_json
# serialize model to JSON
model_json = model.to_json()
with open("/content/drive/My Drive/defi_bigdata/model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/content/drive/My Drive/defi_bigdata/model.h5")
print("Saved model to disk")

In [0]:
# later...
from keras.models import model_from_json
# load json and create model
json_file = open('/content/drive/My Drive/defi_bigdata/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("/content/drive/My Drive/defi_bigdata/model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

In [0]:
X_data_test = np.load("/content/drive/My Drive/defi_bigdata/X_data_test.npy", 'r')
ynew = model.predict_classes(X_data_test)

prediction = pd.DataFrame(ynew, columns=['predictions']).to_csv('prediction.csv', sep = ';')
from google.colab import files
files.download("prediction.csv")